In [1]:
import numpy as np
import cv2

In [2]:
# Input imgs path
labeled_dir = "./new-labeled.pgm"
campus_dir = "./new-campus.pgm"
text_labels_dir = "./new-table.txt"

NUM_BUILDING = 26

In [3]:
# Read in Labels and Image. In grayscale
campus = cv2.imread(campus_dir, 0)
labeled = cv2.imread(labeled_dir, 0)

# building color values & names
b_values = []
b_names = []
label_input = open(text_labels_dir, 'r')
for i in range(0, NUM_BUILDING):
    tmp = label_input.readline()
    split = tmp.split(" ")
    b_values.append(int(split[len(split) - 2]))
    b_names.append(split[len(split) - 1])

In [4]:
# Step 1: Describing shape, size, orientation,
HEIGHT = labeled.shape[0]
WIDTH = labeled.shape[1]

# Record area, min & max coordinates (bounding box)
AREA = np.zeros(NUM_BUILDING)
COM = np.zeros((NUM_BUILDING, 2))
BBOX = np.zeros((NUM_BUILDING, 4))
DESC = [[] for i in range(0, NUM_BUILDING)]

# Iterate through buildings
for i in range(0, NUM_BUILDING):
    # Get the list of coordinates for this building
    pos_list = np.where(labeled == b_values[i])
    
    AREA[i] = int(len(pos_list[0]))
    
    BBOX[i] = [max(pos_list[0]), max(pos_list[1]), min(pos_list[0]), min(pos_list[1])]
    
    COM[i] = [sum(pos_list[1])/AREA[i], sum(pos_list[0])/AREA[i]]

# SIZE: LARGE, MEDIUM, SMALL
maxA = max(AREA)

for i in range(0, NUM_BUILDING):
    
    if AREA[i] > .67 * maxA:
        DESC[i].append('LARGE')
    elif AREA[i] > .33 * maxA:
        DESC[i].append('MEDIUM')
    elif AREA[i] > .1 * maxA:
        DESC[i].append('SMALL')
    else:
        DESC[i].append('TINY')

max_id = np.argmax(AREA)
min_id = np.argmin(AREA)
DESC[min_id].append('SMALLEST')
DESC[max_id].append('LARGEST')

# ORIENTATION: VERTICAL, HORIZONTAL, SYMMETRICAL ORIENTATION

# SHAPE: SQUARE, RECTANGULAR, NON-RECTANGULAR, SYMMETRICAL, NON-SYMMETRICAL

for i in range(0, NUM_BUILDING):
    
    bbox_width = BBOX[i][1] - BBOX[i][3]
    bbox_height = BBOX[i][0] - BBOX[i][2]
    
    bbox_a = bbox_width * bbox_height
    
    if bbox_height > 1.25 * bbox_width:
        DESC[i].append('VERTICAL')
    elif bbox_width > 1.25 * bbox_height:
        DESC[i].append('HORIZONTAL')
    else:
        DESC[i].append('SQUARELY ORIENTED')
        
    # SQUARE OR REC
    if (bbox_a <= 1.02 * AREA[i]):
        if (bbox_width >= 0.9 * bbox_height) and (bbox_width <= 1.1 * bbox_height):
            DESC[i].append('SQUARE')
        else:
            DESC[i].append('RECTANGULAR')
    else:
        DESC[i].append('NON-RECTANGULAR')
        bbox_avg = [(BBOX[i][1] + BBOX[i][3]) / 2, (BBOX[i][0] + BBOX[i][2]) / 2]
        if (COM[i][0] <= 1.005 * bbox_avg[0]) and \
            (COM[i][0] >= 0.995 * bbox_avg[0]) and \
            (COM[i][1] <= 1.005 * bbox_avg[1]) and \
            (COM[i][1] >= 0.995 * bbox_avg[1]):
            DESC[i].append('SYMMETRICAL')
        else:
            DESC[i].append('NOT SYMMETRICAL ON BOTH ORIENTATIONS')



In [5]:
AREA


array([1640., 1435., 5831., 1998., 5753., 3911., 3613.,  322., 1164.,
       1182., 1191., 3898., 1087.,  759., 1307., 1085.,  340.,  225.,
       1590., 1470., 4950., 2615., 5855., 2940., 5282., 1540.])

In [6]:
# Step 2: Describle Absolute Space
DESC2 = [[] for i in range(0, NUM_BUILDING)]

for i in range(0, NUM_BUILDING):
    
    if (COM[i][0] <= 0.33 * WIDTH):
        if (COM[i][1] <= 0.33 * HEIGHT):
            DESC2[i].append("NORTHWEST")
            if (COM[i][1] <= 0.167 * HEIGHT):
                DESC2[i].append("UPPER")
            else:
                DESC2[i].append("LOWER")
        elif (COM[i][1] <= 0.67 * HEIGHT):
            DESC2[i].append("WEST")
            if (COM[i][1] <= 0.5 * HEIGHT):
                DESC2[i].append("UPPER")
            else:
                DESC2[i].append("LOWER")
        else:
            DESC2[i].append("SOUTHWEST")
            if (COM[i][1] <= 0.84 * HEIGHT):
                DESC2[i].append("UPPER")
            else:
                DESC2[i].append("LOWER")
    elif (COM[i][0] <= 0.67 * WIDTH):
        if (COM[i][1] <= 0.33 * HEIGHT):
            DESC2[i].append("NORTH")
            if (COM[i][1] <= 0.167 * HEIGHT):
                DESC2[i].append("UPPER")
            else:
                DESC2[i].append("LOWER")
        elif (COM[i][1] <= 0.67 * HEIGHT):
            DESC2[i].append("CENTER")
            if (COM[i][1] <= 0.5 * HEIGHT):
                DESC2[i].append("UPPER")
            else:
                DESC2[i].append("LOWER")
        else:
            DESC2[i].append("SOUTH")
            if (COM[i][1] <= 0.84 * HEIGHT):
                DESC2[i].append("UPPER")
            else:
                DESC2[i].append("LOWER")
    else:
        if (COM[i][1] <= 0.33 * HEIGHT):
            DESC2[i].append("NORTHEAST")
            if (COM[i][1] <= 0.167 * HEIGHT):
                DESC2[i].append("UPPER")
            else:
                DESC2[i].append("LOWER")
        elif (COM[i][1] <= 0.67 * HEIGHT):
            DESC2[i].append("EAST")
            if (COM[i][1] <= 0.5 * HEIGHT):
                DESC2[i].append("UPPER")
            else:
                DESC2[i].append("LOWER")
        else:
            DESC2[i].append("SOUTHEAST")
            if (COM[i][1] <= 0.84 * HEIGHT):
                DESC2[i].append("UPPER")
            else:
                DESC2[i].append("LOWER")


In [7]:
DESC2

[['NORTHWEST', 'UPPER'],
 ['NORTH', 'UPPER'],
 ['NORTHEAST', 'UPPER'],
 ['NORTHWEST', 'UPPER'],
 ['NORTH', 'LOWER'],
 ['NORTHEAST', 'LOWER'],
 ['NORTHWEST', 'LOWER'],
 ['NORTH', 'LOWER'],
 ['EAST', 'UPPER'],
 ['EAST', 'UPPER'],
 ['WEST', 'UPPER'],
 ['CENTER', 'UPPER'],
 ['EAST', 'UPPER'],
 ['WEST', 'UPPER'],
 ['WEST', 'LOWER'],
 ['EAST', 'LOWER'],
 ['EAST', 'LOWER'],
 ['CENTER', 'LOWER'],
 ['WEST', 'LOWER'],
 ['EAST', 'LOWER'],
 ['CENTER', 'LOWER'],
 ['SOUTHWEST', 'UPPER'],
 ['SOUTHEAST', 'LOWER'],
 ['SOUTHWEST', 'LOWER'],
 ['SOUTH', 'LOWER'],
 ['SOUTHWEST', 'LOWER']]

In [8]:
# Step 3: Describing Relative Space
# T is north of S
def N(S, T):
    if isinstance(T, int) and isinstance(S, int):
        if abs(COM[T][1] - COM[S][1]) < abs(COM[T][0] - COM[S][0]):
            return False
    if isinstance(T, int):
        maxyT = BBOX[T][0]
    else:
        maxyT = T[1]
    if isinstance(S, int):
        minyS = BBOX[S][2]
    else:
        minyS = S[1]
    return maxyT < minyS

    
def E(S, T):
    if isinstance(T, int) and isinstance(S, int):
        if abs(COM[T][1] - COM[S][1]) > abs(COM[T][0] - COM[S][0]):
            return False
    if isinstance(T, int):
        minxT = BBOX[T][3]
    else:
        minxT = T[0]
    if isinstance(S, int):
        maxxS = BBOX[S][1]
    else:
        maxxs = S[0]
    return minxT > maxxS

def W(S, T):
    if isinstance(T, int) and isinstance(S, int):
        if abs(COM[T][1] - COM[S][1]) > abs(COM[T][0] - COM[S][0]):
            return False
    if isinstance(T, int):
        maxxT = BBOX[T][1]
    else:
        maxxT = T[0]
    if isinstance(S, int):
        minxS = BBOX[S][3]
    else:
        minxS = S[0]
    return maxxT < minxS

def S(S, T):
    if isinstance(T, int) and isinstance(S, int):
        if abs(COM[T][1] - COM[S][1]) < abs(COM[T][0] - COM[S][0]):
            return False
    if isinstance(T, int):
        minyT = BBOX[T][2]
    else:
        minyT = T[1]
    if isinstance(S, int):
        maxyS = BBOX[S][0]
    else:
        maxyS = S[1]
    return minyT > maxyS

In [9]:
def Near(S, T):
    bwidth = BBOX[S][1] - BBOX[S][3]
    bheight = BBOX[S][0] - BBOX[S][2]
    minx = int(COM[S][0] - bwidth)
    maxx = int(COM[S][0] + bwidth)
    miny = int(COM[S][1] - bheight)
    maxy = int(COM[S][1] + bheight)
    if minx < 0:
        minx = 0
    if maxx >= WIDTH:
        maxx = WIDTH - 1
    if miny < 0:
        miny = 0
    if maxy >= HEIGHT:
        maxy = HEIGHT - 1
    # S is always a building.
    if isinstance(T, int):
        if DESC2[T] == DESC2[S]:
            return True
        else:
            area = labeled[miny:maxy][:,minx:maxx]
            overlap = np.where(area == b_values[T])
            if np.size(overlap) == 0:
                return False
            return True
    else:
        if T[0] >= minx and T[0] <= maxx and T[1] >= miny and T[1] <= maxy:
            return True
        else:
            return False

In [10]:
NORTH_M = np.zeros((NUM_BUILDING, NUM_BUILDING), bool)
SOUTH_M = np.zeros((NUM_BUILDING, NUM_BUILDING), bool)
WEST_M = np.zeros((NUM_BUILDING, NUM_BUILDING), bool)
EAST_M = np.zeros((NUM_BUILDING, NUM_BUILDING), bool)
NEAR_M = np.zeros((NUM_BUILDING, NUM_BUILDING), bool)

for i in range(0, NUM_BUILDING):
    for j in range(0, NUM_BUILDING):
        if i != j:
            NORTH_M[i][j] = N(i, j)
            SOUTH_M[i][j] = S(i, j)
            WEST_M[i][j] = W(i ,j)
            EAST_M[i][j] = E(i, j)
            NEAR_M[i][j] = Near(i, j)

for j in range(0, NUM_BUILDING):
    for i in range(0, NUM_BUILDING):
        if NORTH_M[i][j]:
            for k in range(0, NUM_BUILDING):
                if NORTH_M[j][k]:
                    NORTH_M[i][k] = False
        if SOUTH_M[i][j]:
            for k in range(0, NUM_BUILDING):
                if SOUTH_M[j][k]:
                    SOUTH_M[i][k] = False
        if WEST_M[i][j]:
            for k in range(0, NUM_BUILDING):
                if WEST_M[j][k]:
                    WEST_M[i][k] = False
        if EAST_M[i][j]:
            for k in range(0, NUM_BUILDING):
                if EAST_M[j][k]:
                    EAST_M[i][k] = False

for i in range(0, NUM_BUILDING):
    for j in range(0, NUM_BUILDING):
        if NORTH_M[i][j] and SOUTH_M[j][i]:
            SOUTH_M[j][i] = False
        if EAST_M[i][j] and WEST_M[j][i]:
            WEST_M[j][i] = False

In [11]:
# Print Step 3 results:
v = 0
h = 0
n = 0
for i in range(0, NUM_BUILDING):
    for j in range(0, NUM_BUILDING):
        if NORTH_M[i][j]:
            print(b_names[i] + "--North-->" + b_names[j])
            v = v + 1
        if SOUTH_M[i][j]:
            print(b_names[i] + "--South-->" + b_names[j])
            v = v + 1
        if EAST_M[i][j]:
            print(b_names[i] + "--East-->" + b_names[j])
            h = h + 1
        if WEST_M[i][j]:
            print(b_names[i] + "--West-->" + b_names[j])
            h = h + 1
        if NEAR_M[i][j]:
            print(b_names[i] + "--Near-->" + b_names[j])
            n = n + 1
            

Pupin
--East-->SchapiroCEPSR

Pupin
--Near-->SchapiroCEPSR

Pupin
--Near-->NorthwestCorner

Pupin
--East-->Schermerhorn

Pupin
--East-->Fayerweather

SchapiroCEPSR
--Near-->Pupin

SchapiroCEPSR
--East-->Mudd&EngTerrace&Fairchild&CS

SchapiroCEPSR
--Near-->Mudd&EngTerrace&Fairchild&CS

SchapiroCEPSR
--Near-->Uris

Mudd&EngTerrace&Fairchild&CS
--Near-->SchapiroCEPSR

Mudd&EngTerrace&Fairchild&CS
--Near-->Uris

Mudd&EngTerrace&Fairchild&CS
--Near-->Schermerhorn

NorthwestCorner
--East-->Pupin

NorthwestCorner
--Near-->Pupin

NorthwestCorner
--East-->Uris

NorthwestCorner
--Near-->Chandler&Havemeyer

NorthwestCorner
--East-->Avery

NorthwestCorner
--East-->StPaulChapel

NorthwestCorner
--East-->Philosophy

Uris
--North-->Pupin

Uris
--Near-->Pupin

Uris
--North-->SchapiroCEPSR

Uris
--Near-->SchapiroCEPSR

Uris
--Near-->Mudd&EngTerrace&Fairchild&CS

Uris
--East-->Schermerhorn

Uris
--Near-->Schermerhorn

Uris
--Near-->Chandler&Havemeyer

Uris
--Near-->ComputerCenter

Uris
--Near-->Avery

U

In [12]:
v


55

In [13]:
# Step 4: User interface
campus_clone = campus.copy()
cv2.namedWindow("campus")

def getdesc(refPt):
    
    desc = ""
    for i in range(0, NUM_BUILDING):
        

def uponclick(event, x, y, flags, param):
    
    global refPt
    
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [x, y]
        
    cloud = getdesc(refPt)

while True:
    cv2.imshow("campus", campus)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("r"):
        #reset
        campus = campus_clone.copy()
    elif key == ord("q"):
        #quit
        break
cv2.destroyAllWindows()